# LLM Mastery Align
Code to Align LLM 

In [2]:
# # If requirements.txt is in the same directory as your notebook
# # !pip install -r requirements.txt
# import torch

# # Check if CUDA is available
# print(f"CUDA available: {torch.cuda.is_available()}")

# # Check CUDA version
# if torch.cuda.is_available():
#     print(f"CUDA version: {torch.version.cuda}")
    
#     # Get the current device
#     current_device = torch.cuda.current_device()
    
#     # Get the name of the current device
#     print(f"GPU device: {torch.cuda.get_device_name(current_device)}")
    
#     # Test a small tensor operation on GPU
#     x = torch.tensor([1.0, 2.0, 3.0]).cuda()
#     print(f"Tensor on GPU: {x}")
#     print(f"Device of tensor: {x.device}")
# else:
#     print("No GPU support detected. PyTorch is using CPU only.")

CUDA available: True
CUDA version: 12.1
GPU device: NVIDIA GeForce RTX 3080
Tensor on GPU: tensor([1., 2., 3.], device='cuda:0')
Device of tensor: cuda:0


In [ ]:
# import requests, zipfile, io
# file_url = 'https://ideami.com/llm_align'
# print ('downloading fiels using python')
# response = requests.get(file_url)
# zipfile.ZipFile(io.BytesIO(response.content)).extractall('.')

downloading fiels using python


SSLError: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2580)

In [ ]:
# !pip install flash-attn
# !pip install datasets

## Import libraries

In [103]:
from datetime import datetime
from datasets import load_dataset, load_from_disk
import ipdb
import math
import os, sys
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import transformers
from typing import List, Dict, Union

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32= True

torch.cuda.empty_cache()

torch.set_printoptions(threshold=10000)




## Training parameters

In [106]:
# trainign paramters
batch_size = 1
epochs = 3
lr = 6e-5
lr_warmup_steps = 100
context = 1024
alpha = 0.5 # weigiting for ORPO
prompt_max_size = 512
compile = False
dtype = torch.bfloat16
log_iters = 50

# Hyperparamters
dropout = 0.
grad_clip = 1.0
weight_decay =  0.0

# DEvice
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print ('device: ', device) 


device:  cuda


In [107]:
# logging
project_name = 'aligntest'
wandb_log = True
wandb_project = project_name
wandb_run_name = 'alignment-run'
wandb_run_name = 'alignment-run'
wandb_run_name = 'alignment-run' + datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

if wandb_log:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name)

In [120]:
dataset_path = './data/orpo_dataset9_3'
dataset_name  = 'mlabonne/orpo-dpo-mix-40k'
tokenizer_path = 'tokenizers/tok16384'
checkpoint_dir = './models/'

# Tokenizer Dataset
tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_path)

# Set our interction template
# tokenizer.chat_template = "{% for message in messages%}{% if messge['role]==user'}\n{{'<|user|> + messge['content] + eos_token'}}\{}
tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

tokenizer.pad_token = tokenizer.eos_token

if os.path.exists(dataset_path):
    dataset = load_from_disk(dataset_path)
else:
    print ('filtering and tokenizing datast')
    dataset = load_dataset(dataset_name, split='all')

    # filter dataset
    # dataset = dataset.filter(lambda r: r["source"] != 'toxic-dpo-v0.2')

    # shorten to     512
    def filter_dataset(examples):
        prompt_length = tokenizer.apply_chat_template(examples['chosen'][:-1], tokenize=True, add_generation_prompt=True, return_tensors='pt').size(-1)
        # prompt_length = tokenizer.apply_chat_template(examples['chosen'][:-1], tokenize=True, add_generation_prompt=True, return_tensors='pt').size(-1)  


        if prompt_length < prompt_max_size:
            return True
        else:
            return False

    # preprocess and tokenize data
    def preprocess_dataset (examples: Union [List, Dict]):
        # select chosen field and releimante last itme
        prompt = [tokenizer.apply_chat_template(item[:-1], tokenize=False, add_generation_prompt=True) for item in examples['chosen']]
        chosen = [tokenizer.apply_chat_template(item, tokenize = False) for item in examples['chosen']]
        rejected = [tokenizer.apply_chat_template(item, tokenize = False) for item in examples['rejected']]

        # tokenizer
        # Hugginf face Dict Format
        # Fielsd: ids, type_id, tokesn, offsets, attention_masks, special_tokens_mask, overflowing
        inputs = tokenizer(prompt, max_length=context, padding='max_length', truncation=True, return_tensors='pt')
        pos_labels = tokenizer(chosen, max_length=context, padding='max_length', truncation=True, return_tensors='pt')
        neg_labels = tokenizer(rejected, max_length=context, padding='max_length', truncation=True, return_tensors='pt')

        # add fields to input
        inputs['positive_input_ids'] = pos_labels['input_ids']
        inputs['positive_attention_mask'] = pos_labels['attention_mask']

        inputs['negative_input_ids'] = neg_labels['input_ids']
        inputs['negative_attention_mask'] = neg_labels['attention_mask']

        return inputs




    dataset = dataset.filter (filter_dataset)

    # pre process and dataset
    # for multiprocessing : num_processing (32, os.cpu_count())
    dataset = dataset.map(preprocess_dataset, batched = True, num_proc=1, remove_columns=dataset.column_names)

    dataset.save_to_disk(dataset_path)

    

filtering and tokenizing datast


Map:   0%|          | 0/39091 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/39091 [00:00<?, ? examples/s]

In [122]:
dataset
# negative_input_id

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'positive_input_ids', 'positive_attention_mask', 'negative_input_ids', 'negative_attention_mask'],
    num_rows: 39091
})

## Split the data

In [123]:
seed = 42
dataset = dataset.shuffle(seed).train_test_split(test_size=0.05)
train_data = dataset['train']
val_data = dataset['test']


In [ ]:
# train_data

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'positive_input_ids', 'positive_attention_mask', 'negative_input_id', 'negative_attenion_mask'],
    num_rows: 37136
})

In [124]:


data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#  Setup DataLoaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=False, collate_fn=data_collator, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_data, batch_size, shuffle=False, collate_fn=data_collator, num_workers=0)

In [125]:
it = iter (train_loader)
batch = next(it)
# print(tokenizer.decode(batch['input_ids']))[0]
print(tokenizer.decode(batch['input_ids'][0]))

<|user|>
An isosceles, obtuse triangle has one angle with a degree measure that is 50$\%$ larger than the measure of a right angle. What is the measure, in degrees, of one of the two smallest angles in the triangle? Express your answer as a decimal to the nearest tenth.</s> 
<|assistant|>
</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

## Set up Architecture

In [126]:
from llm import Llama, ModelArgs

checkpoint = torch.load(os.path.join (checkpoint_dir, 'base_model.pt'))
config = checkpoint.pop('config')

model_args = ModelArgs(
    dim=config.hidden_size,
    n_layers=config.num_hidden_layers,
    n_heads = config.num_attention_heads,
    n_kv_heads = config.num_key_value_heads,
    vocab_size = config.vocab_size,
    norm_eps= config.rms_norm_eps,
    rope_theta=-config.rope_theta,
    max_seq_len=context,
    dropout=config.attention_dropout,
    hidden_dim=config.intermediate_size,
    attention_bias=config.attention_bias,
    mlp_bias = config.mlp_bias

)


model = Llama(model_args)
model.load_state_dict(checkpoint)

model = model.to(dtype)
model = model.to(device)

if compile:
    print ('[INFO] comiling model')
    torch = torch.compile(model)
else:
    print('no compile')

print (sum(p.numel() for p in model.parameters())/1e6, "M parameter")

no compile
138.431232 M parameter


## Optimizer

In [127]:
optimizer = torch.optim.Adam(model.parameters(), lr = lr, betas = (0.9, 0.95), fused = device== 'cuda', weight_decay=weight_decay)

num_traiing_steps = len (train_loader) * epochs

def lr_lambda (current_step):
    if current_step < lr_warmup_steps:
        return float(current_step) /float(max( 1, lr_warmup_steps))
    progress = float(current_step - lr_warmup_steps) /float(max(1, num_traiing_steps - lr_warmup_steps))
    return max (0.0, 0.5 * (1.0 + math.cos(math.pi + float(0.5)* 2.0 * progress)))

scheduler = torch.optim.lr_scheduler.LambdaLR (optimizer, lr_lambda, last_epoch = -1)


In [128]:
def compute_logps(prompt_attention_mask, chosen_inputs, chosen_attention_mask, logits):
    
    mask = chosen_attention_mask[:, : -1] - prompt_attention_mask[:, 1:]
    per_token_logps = torch.gather(logits[:, : -1, :].log_softmax(-1), dim = 2,
                                    index= (mask * chosen_inputs[:,1:]).unsqueeze(2)).squeeeze(2)
    
    return torch.mul(per_token_logps, mask.to(dtype)).sum(dim = 1).to (dtype)/ mask.sum(dim = 1).to(dtype)

In [ ]:
# p_mask =    torch.tensor ([1,1,1,1,0,0,0,0,0,0,0])
# c_mask =    torch.tensor ([1,1,1,1,1,1,1,0,0,0,0])
# c_inputs =  torch.tensor ([2,4,3,2,4,2,1,0,0,0,0])
# print(f"p_mask: {p_mask}")
# print(f"c_inputs: {c_inputs}")
# print(f"c_mask: {c_mask}")
# mask = c_mask[:-1] - p_mask[1:]
# print(f"mask: {mask}")

# logits = torch.tensor([
#     [0.2, 0.4, 0.8, 0.1, 0.3],
#     [0.2, 0.1, 0.5, 0.12, 0.31],
#     [0.22, 0.44, 0.81, 0.13, 0.32],
#     [0.29, 0.42, 0.84, 0.15, 0.32],
#     [0.24, 0.48, 0.88, 0.17, 0.34],
#     [0.21, 0.41, 0.81, 0.14, 0.33],
#     [0.23, 0.43, 0.82, 0.16, 0.35],
#     [0.2, 0.4, 0.8, 0.1, 0.3],
#     [0.2, 0.1, 0.5, 0.12, 0.31],
#     [0.22, 0.44, 0.81, 0.13, 0.32],
#     [0.22, 0.44, 0.81, 0.13, 0.32]
# ])

# print(f"c_inputs[1:]: {c_inputs[1:]}")
# index=(mask * c_inputs[1:])
# print(f"index: {index}")

# print (index.shape, logits.shape)

# # Expand dimensions for correct gather shape
# index_expanded = index.unsqueeze(1)
# print(f"index_expanded: {index_expanded}")
# print (index_expanded.shape, logits.shape)

# # gather the values at the specified indicies
# gathered_values = torch.gather(logits[:-1, :], dim = 1, index=index_expanded)
# print (f'gathered values: {gathered_values}')

# # remove extr dimnstion
# per_token_logps = gathered_values.squeeze(1)
# print (f'per token_logps> {per_token_logps}')

# result = torch.mul (per_token_logps, mask)
# print (f'result: {result}')
# f1 = result.sum(dim = 0)
# f2 = mask.sum(dim =0)
# print (f'f1: {f1}')
# print (f'f2: {f2}')
# final = f1/f2
# print (f'fina: {final}')

p_mask: tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])
c_inputs: tensor([2, 4, 3, 2, 4, 2, 1, 0, 0, 0, 0])
c_mask: tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])
mask: tensor([0, 0, 0, 1, 1, 1, 1, 0, 0, 0])
c_inputs[1:]: tensor([4, 3, 2, 4, 2, 1, 0, 0, 0, 0])
index: tensor([0, 0, 0, 4, 2, 1, 0, 0, 0, 0])
torch.Size([10]) torch.Size([11, 5])
index_expanded: tensor([[0],
        [0],
        [0],
        [4],
        [2],
        [1],
        [0],
        [0],
        [0],
        [0]])
torch.Size([10, 1]) torch.Size([11, 5])
gathered values: tensor([[0.2000],
        [0.2000],
        [0.2200],
        [0.3200],
        [0.8800],
        [0.4100],
        [0.2300],
        [0.2000],
        [0.2000],
        [0.2200]])
per token_logps> tensor([0.2000, 0.2000, 0.2200, 0.3200, 0.8800, 0.4100, 0.2300, 0.2000, 0.2000,
        0.2200])
result: tensor([0.0000, 0.0000, 0.0000, 0.3200, 0.8800, 0.4100, 0.2300, 0.0000, 0.0000,
        0.0000])
f1: 1.840000033378601
f2: 4
fina: 0.46000000834465027


## Alignment Training Loop

In [133]:
# Alignment Training LookupError

try: 
    for e in range (epochs):
        for i, batch in tqdm (enumerate(train_loader), total=len(train_loader), dynamic_ncols=True):
            optimizer.zero_grad(set_to_none=True)

            batch['positive_input_ids'] = batch['positive_input_ids'].to(device)
            batch['positive_attention_mask'] = batch['positive_attention_mask'].to(device)
            batch['negative_input_ids'] = batch['negative_input_ids'].to(device)
            batch['negative_attention_mask'] = batch['negative_attention_mask'].to(device)
            batch['attention_mask'] = batch['attention_mask'].to(device)

            neg_lables = batch['negative_input_ids'].clone()
            pos_labels = batch['positive_input_ids'].clone()

            # mask will have is prompat
            mask = batch['attention_mask'] *batch['positive_attention_mask']
            pos_labels = pos_labels * mask.logical_not()
            

            pos_labels[pos_labels==0] = tokenizer.pad_token_id
            pos_labels[pos_labels == tokenizer.pad_token_id] = - 100
            neg_lables[neg_lables == tokenizer.pad_token_id] = - 100

            outputs_pos, loss_pos = model(batch['positive_input_ids'], pos_labels)
            outputs_neg, loss_neg = model(batch['negative_input_ids'], neg_lables)

            # calcute per toaken log probability needed to calcualte OPRO LOG ODDs

            pos_prob = compute_logps(
                prompt_attention_mask=batch['attention_mask'],
                chosen_inputs=batch['positive_input_ids'], 
                chosen_attention_mask=batch['positive_attention_mask'],
                logits = outputs_pos
            )

            neg_prob = compute_logps(
                prompt_attention_mask=batch['attention_mask'],
                chosen_inputs=batch['negative_input_ids'],
                chosen_attention_mask=batch['negative_attention_mask'],
                logits = outputs_neg

            )

            # calcuate oprs ods ratio
            log_odds = (pos_prob - neg_prob) - (torch.log (1 - torch.exp(pos_prob)) - torch.log(1 - torch.exp(neg_prob)))
            # log_odds = (pos_prob - neg_prob) - (torch.log (1 - torch.exp(pos_prob)) - torch.log(1 - torch.exp(neg_prob)))
            sig_ratio = F.sigmoid(log_odds)
            ratio = torch.log(sig_ratio)

            loss = torch.mean  (loss_pos - (alpha*ratio).mean()).to(dtype=dtype)

            # logging
            if i%log_iters == 0:
                # calcuale averagelos

                # loss_m, log_iters_m, ratio_m = calculate_loss()

                print(f"Epochs [{e}/{epochs}] Step: [{i}/{len(train_loader)}], train loss: {loss_m['train']:.4f}, val loss: {loss_m['val']:.4f}, Odds Ratio: {log_odds_m['train']:.4f}, val Odds Ratio: {log_odds_m['val']:.4f}")

                if wandb_log:
                    wandb.log({
                        # "train_loss": loss_m['train'],
                        # "val_loss": loss_m['val'],
                        # "train_log_odds": log_odds_m['train'],
                        # "val_log_odds": log_odds_m['val'],
                        # "train_ratio": (alpha*ratio_m['train']),
                        # "val_ratio": (alpha*ratio_m['val']),
                        "pos_prob": pos_prob.mean().item(),
                        "neg_prob": neg_prob.mean().item(),                        
                        "lr": scheduler.get_last_lr()[0],
                    }, 
                    step = (e*len(train_loader) + i))

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)
            optimizer.step()
            scheduler.step()

        # save checkpoint
        sd = model.state_dict()
        sd['config'] = config
        torch.save(sd, os.path.join(checkpoint_dir, f'{project_name}_{e+1}.pt'))





except KeyboardInterrupt:
    print ('Training eroor, cleaning up')

finally:
    # release GPU memory
    torch.cuda.empty_cache()
    print ('GPU Memory released')
    sys.exit(0)


  0%|          | 0/37136 [00:00<?, ?it/s]


GPU Memory released


AttributeError: 'tuple' object has no attribute 'tb_frame'